In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
from tensorflow import math as tfm
#import tensorflow_probability as tfp
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
#import cartopy.crs as ccrs
import matplotlib.ticker as mticker
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
import sklearn
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from cbrain.imports import *
from cbrain.utils import *
from cbrain.normalization import *
import h5py
from sklearn.preprocessing import OneHotEncoder
from cbrain.climate_invariant import *
import yaml
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

ImportError: This version of TensorFlow Probability requires TensorFlow version >= 2.3; Detected an installation of version 2.0.0. Please upgrade TensorFlow to proceed.

## DataGenerator

In [2]:
# Load coordinates (just pick any file from the climate model run)
coor = xr.open_dataset("/DFS-L/DATA/pritchard/ankitesg/data/CESM2_f19_v13_updated_NN_pelayout01_ens_07.cam.h1.2003-01-22-00000.nc",\
                    decode_times=False)
lat = coor.lat; lon = coor.lon; lev = coor.lev;
DATA_DIR = '/DFS-L/DATA/pritchard/ankitesg/datav4/'
hyam = coor.hyam
hybm = coor.hybm
scale_dict = load_pickle('/export/nfs0home/ankitesg/tom/CBRAIN-CAM/nn_config/scale_dicts/2020_10_16_scale_dict_RG.pkl')['scale_dict_RG']

### RH Gen

In [18]:
in_vars_RH = ['RH','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars_RH =['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

TRAINFILE_RH = 'RG_RH_M4K_NORM_train_shuffle.nc'
NORMFILE_RH = 'RG_RH_M4K_NORM_norm.nc'
VALIDFILE_RH = 'RG_RH_M4K_NORM_valid.nc'
BASE_DIR = '/DFS-L/DATA/pritchard/ankitesg/'

In [19]:
train_gen_RH = DataGenerator(
    data_fn = f"{BASE_DIR}datav4/{TRAINFILE_RH}",
    input_vars = in_vars_RH,
    output_vars = out_vars_RH,
    norm_fn = f"{BASE_DIR}datav4/{NORMFILE_RH}",
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True
)

In [20]:
train_gen_RH[0][0].shape

(1024, 108)

In [21]:
class DataGeneratorClimInvRealGeo(DataGenerator):

    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None, normalize_flag=True,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train', exp=None):
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.exp = exp
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off,normalize_flag) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        if self.rh_trans:
            self.qv2rhLayer = QV2RHNumpyReal(self.inp_sub,self.inp_div,inp_subRH,inp_divRH,hyam,hybm)
            


    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size

        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]

        # Split into inputs and outputs
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        X_result = X_norm
        Y = self.output_transform.transform(Y)
        if self.rh_trans:
            X_result = self.qv2rhLayer.process(X_result)
    
        return X_result, Y
    

In [22]:
in_vars = ['QBP','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

In [23]:
TRAINFILE = 'RG_SP_M4K_train_shuffle.nc'
NORMFILE = 'RG_SP_M4K_NORM_norm.nc'
VALIDFILE = 'RG_SP_M4K_valid.nc'

In [24]:
train_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = True,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [25]:
valid_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{VALIDFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = True,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [18]:
train_gen[0][0]

array([[-0.00532146, -0.0147291 , -0.01748944, ..., -0.23045476,
         0.00330495,  0.01967537],
       [-0.00503114, -0.01454457, -0.01742817, ..., -0.19863862,
        -0.03660068, -0.0278144 ],
       [ 0.0013068 ,  0.00793262,  0.07647736, ..., -0.23045476,
        -0.04269275, -0.04149439],
       ...,
       [-0.00542634, -0.01501025, -0.01805928, ..., -0.08419288,
        -0.04737879, -0.04183535],
       [-0.00341021, -0.01143109, -0.01489481, ..., -0.23045476,
        -0.01086429, -0.03869757],
       [-0.00451332, -0.01299936, -0.0163122 , ..., -0.23045476,
         0.00899058, -0.02446111]], dtype=float32)

## RH Network

In [26]:
model = Sequential()
model.add(Input(shape=(108,)))
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.2))
for i in range (4):
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
for i in range (3):
    model.add(Dense(256, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
model.add(Dense(112, activation='linear'))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               13952     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 128)              

In [28]:
opt = tf.keras.optimizers.Adam()

In [29]:
model.compile(optimizer=opt, loss='mse')

In [30]:
model.load_weights(path_HDF5+'RH_RGV1.h5')

In [31]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'RH_RGV1.h5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
with tf.device('/gpu:0'):
    Nep = 15
    model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
                  callbacks=[earlyStopping, mcp_save])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
92945/92947 [============================>.] - ETA: 0s - batch: 46472.0000 - size: 1024.0000 - loss: 225.5949- ETA: 3s - batch: 46431.5000 - WARNING:tensorflow:From /export/home/ankitesg/anaconda3/envs/CBrainEnv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
92947/92947 [==============================] - 4837s 52ms/step - batch: 46473.0000 - size: 1024.0000 - loss: 225.5938 - val_loss: 202.2278
Epoch 2/15
92947/92947 [==============================] - 5691s 61ms/step - batch: 46473.0000 - size: 1024.0000 - loss: 198.7253 - val_loss: 221.2858
Epoch 3/15
69321/92947 [=====================>........] - ETA: 16:43 - batch: 34660.0000 - si

In [ ]:
with tf.device('/gpu:0'):
    Nep = 15
    model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
                  callbacks=[earlyStopping, mcp_save])

Epoch 1/15
92946/92947 [============================>.] - ETA: 0s - batch: 46472.5000 - size: 1024.0000 - loss: 198.7802WARNING:tensorflow:From /export/home/ankitesg/anaconda3/envs/CBrainEnv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
92947/92947 [==============================] - 3537s 38ms/step - batch: 46473.0000 - size: 1024.0000 - loss: 198.7802 - val_loss: 217.1007
Epoch 2/15
92947/92947 [==============================] - 1325s 14ms/step - batch: 46473.0000 - size: 1024.0000 - loss: 192.6193 - val_loss: 244.1864
Epoch 3/15
92947/92947 [==============================] - 6014s 65ms/step - batch: 46473.0000 - size: 1024.0000 - loss: 189.1395 - val_loss: 273.5659
Epoch 4/15
92947/92947 [===========================

## RH and QBP both Models

In [35]:
in_vars = ['QBP','RELHUM','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

In [36]:
train_gen = DataGeneratorClimInv(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)


In [38]:
valid_gen = DataGeneratorClimInv(
    data_fn = f'{DATA_DIR}{VALIDFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)


In [39]:
model = Sequential()
model.add(Input(shape=(134,)))
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.2))
for i in range (4):
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
for i in range (3):
    model.add(Dense(256, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
model.add(Dense(112, activation='linear'))

In [40]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 128)               17280     
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 128)              

In [41]:
opt = tf.keras.optimizers.Adam()

In [42]:
model.compile(optimizer=opt, loss='mse')

In [44]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'RH_RGV2.h5',save_best_only=True, monitor='val_loss', mode='min')

In [46]:
with tf.device('/gpu:0'):
    Nep = 15
    model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
                  callbacks=[earlyStopping, mcp_save])

Epoch 1/15
92947/92947 [==============================] - 5285s 57ms/step - batch: 46473.0000 - size: 1024.0000 - loss: 158.0118 - val_loss: 99.3919
Epoch 2/15
92947/92947 [==============================] - 2208s 24ms/step - batch: 46473.0000 - size: 1024.0000 - loss: 110.3213 - val_loss: 133.8753
Epoch 3/15
92947/92947 [==============================] - 2868s 31ms/step - batch: 46473.0000 - size: 1024.0000 - loss: 99.3966 - val_loss: 153.3458
Epoch 4/15
37871/92947 [===========>..................] - ETA: 12:53 - batch: 18935.0000 - size: 1024.0000 - loss: 94.9165

KeyboardInterrupt: 